In [1]:
import numpy as np

In [2]:
class LARS_Lasso:
    def __init__(self, alpha=1.0, max_iter=500, tol=1e-4):
        self.alpha = alpha
        self.max_iter = max_iter
        self.tol = tol
        self.coef_ = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.coef_ = np.zeros(n_features)
        residual = y.copy()
        active_set = []
        inactive_set = list(range(n_features))

        for iteration in range(self.max_iter):
            # Compute correlations
            correlations = np.dot(X.T, residual)

            # Find the maximum correlation
            max_corr = np.max(np.abs(correlations[inactive_set]))
            if max_corr < self.tol:
                break

            # Identify the variable to enter the active set
            max_corr_index = np.argmax(np.abs(correlations[inactive_set]))
            j = inactive_set[max_corr_index]
            active_set.append(j)
            inactive_set.remove(j)

            # Compute the step size
            sign_j = np.sign(correlations[j])
            step_size = (max_corr - self.alpha) / np.dot(X[:, j].T, X[:, j])

            # Update coefficients and residual
            self.coef_[j] += step_size * sign_j
            residual -= step_size * sign_j * X[:, j]

            # Check for convergence
            if np.max(np.abs(residual)) < self.tol:
                break

        return self

    def predict(self, X):
        return np.dot(X, self.coef_)

In [3]:
# Example usage:
if __name__ == "__main__":
    from sklearn.datasets import make_regression
    from sklearn.preprocessing import StandardScaler

    # Generate a random regression problem
    X, y = make_regression(n_samples=100, n_features=20, noise=0.1, random_state=42)

    # Standardize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Fit LARS Lasso model
    model = LARS_Lasso(alpha=0.1)
    model.fit(X, y)

    # Predict using the model
    predictions = model.predict(X)

    print("Coefficients:", model.coef_)
    print("Predictions:", predictions[:10])


ValueError: zero-size array to reduction operation maximum which has no identity